# Speculative Decoding

SGLang now provides an EAGLE-based (EAGLE-2/EAGLE-3) speculative decoding option. Our implementation aims to maximize speed and efficiency and is considered to be among the fastest in open-source LLM engines.

### Performance Highlights

Please see below for the huge improvements on throughput for LLaMA-Instruct 3.1 8B tested on MT bench that can be achieved via EAGLE3 decoding.
For further details please see the [EAGLE3 paper](https://arxiv.org/pdf/2503.01840).

| Method | Throughput (tokens/s) |
|--------|----------------|
| SGLang (w/o speculative, 1x H100) | 158.34 tokens/s |
| SGLang + EAGLE-2 (1x H100) | 244.10 tokens/s |
| SGLang + EAGLE-3 (1x H100) | 373.25 tokens/s |

## EAGLE Decoding

To enable EAGLE speculative decoding the following parameters are relevant:
* `speculative_draft_model_path`: Specifies draft model. This parameter is required.
* `speculative_num_steps`: Depth of autoregressive drafting. Increases speculation range but risks rejection cascades. Default is 5.
* `speculative_eagle_topk`: Branching factor per step. Improves candidate diversity, will lead to higher acceptance rate, but more lead to higher memory/compute consumption. Default is 4.
* `speculative_num_draft_tokens`: Maximum parallel verification capacity. Allows deeper tree evaluation but will lead to higher GPU memory usage. Default is 8.

These parameters are the same for EAGLE-2 and EAGLE-3.

You can find the best combinations of these parameters with [bench_speculative.py](https://github.com/sgl-project/sglang/blob/main/scripts/playground/bench_speculative.py).

In the documentation below, we set `--cuda-graph-max-bs` to be a small value for faster engine startup. For your own workloads, please tune the above parameters together with `--cuda-graph-max-bs`, `--max-running-requests`, `--mem-fraction-static` for the best performance. 

### EAGLE-2 decoding

You can enable EAGLE-2 decoding by setting `--speculative_algorithm EAGLE` and choosing an appropriate model.

In [1]:
from sglang.test.doc_patch import launch_server_cmd
from sglang.utils import wait_for_server, print_highlight, terminate_process

import openai

In [2]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-2-7b-chat-hf  --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-llama2-chat-7B --speculative-num-steps 3 \
    --speculative-eagle-topk 4 --speculative-num-draft-tokens 16 --cuda-graph-max-bs 8
"""
)

wait_for_server(f"http://localhost:{port}")

W0817 00:27:35.446000 2971837 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0817 00:27:35.446000 2971837 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-17 00:27:36] server_args=ServerArgs(model_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='127.0.0.1', port=35964, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='auto', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.849, max_running_requests=128, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=23767

[2025-08-17 00:27:37] Using default HuggingFace chat template with detected content format: string


W0817 00:27:47.116000 2972646 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0817 00:27:47.116000 2972646 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


W0817 00:27:47.571000 2972647 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0817 00:27:47.571000 2972647 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


...

Capturing batches (bs=1 avail_mem=32.49 GB):  75%|███████▌  | 3/4 [00:05<00:01,  1.43s/it]

Capturing batches (bs=1 avail_mem=32.49 GB): 100%|██████████| 4/4 [00:09<00:00,  2.40s/it]
[2025-08-17 00:28:09] Capture draft cuda graph end. Time elapsed: 10.13 s. mem usage=-17.07 GB. avail mem=49.77 GB.
[2025-08-17 00:28:09] Capture draft extend cuda graph begin. This can take up to several minutes. avail mem=49.77 GB


Capturing batches (bs=1 avail_mem=49.65 GB): 100%|██████████| 4/4 [00:00<00:00, 117.71it/s]
[2025-08-17 00:28:10] Capture draft extend cuda graph end. Time elapsed: 0.60 s. mem usage=0.12 GB. avail mem=49.65 GB.
[2025-08-17 00:28:10] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=128, context_len=4096, available_gpu_mem=49.65 GB
[2025-08-17 00:28:10] INFO:     Started server process [2971837]
[2025-08-17 00:28:10] INFO:     Waiting for application startup.
[2025-08-17 00:28:10] INFO:     Application startup complete.
[2025-08-17 00:28:10] INFO:     Uvicorn running on http://127.0.0.1:35964 (Press CTRL+C to quit)


[2025-08-17 00:28:10] INFO:     127.0.0.1:42560 - "GET /v1/models HTTP/1.1" 200 OK


[2025-08-17 00:28:11] INFO:     127.0.0.1:42562 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-17 00:28:11] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-08-17 00:28:11] INFO:     127.0.0.1:42572 - "POST /generate HTTP/1.1" 200 OK
[2025-08-17 00:28:11] The server is fired up and ready to roll!


In [3]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Llama-2-7b-chat-hf",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-08-17 00:28:15] Prefill batch. #new-seq: 1, #new-token: 16, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-08-17 00:28:15] INFO:     127.0.0.1:39042 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [4]:
terminate_process(server_process)

### EAGLE-2 Decoding with `torch.compile`

You can also enable `torch.compile` for further optimizations and optionally set `--torch-compile-max-bs`:


In [5]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-2-7b-chat-hf  --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-llama2-chat-7B --speculative-num-steps 5 \
        --speculative-eagle-topk 8 --speculative-num-draft-tokens 64 --mem-fraction 0.6 \
            --enable-torch-compile --torch-compile-max-bs 2
"""
)

wait_for_server(f"http://localhost:{port}")

W0817 00:28:23.113000 2976151 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0817 00:28:23.113000 2976151 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-17 00:28:24] server_args=ServerArgs(model_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='127.0.0.1', port=32872, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='auto', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.6, max_running_requests=128, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=8162666

[2025-08-17 00:28:24] Using default HuggingFace chat template with detected content format: string


W0817 00:28:31.569000 2976683 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0817 00:28:31.569000 2976683 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.
W0817 00:28:31.569000 2976682 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0817 00:28:31.569000 2976682 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-17 00:28:32] Attention backend not explicitly specified. Use flashinfer backend by default.
[2025-08-17 00:28:32] Init torch distributed begin.


...

Capturing batches (bs=1 avail_mem=52.84 GB): 100%|██████████| 4/4 [00:00<00:00, 27.52it/s]
[2025-08-17 00:29:41] Capture draft extend cuda graph end. Time elapsed: 1.80 s. mem usage=0.61 GB. avail mem=52.84 GB.
[2025-08-17 00:29:41] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=128, context_len=4096, available_gpu_mem=52.84 GB


[2025-08-17 00:29:41] INFO:     Started server process [2976151]
[2025-08-17 00:29:41] INFO:     Waiting for application startup.
[2025-08-17 00:29:41] INFO:     Application startup complete.
[2025-08-17 00:29:41] INFO:     Uvicorn running on http://127.0.0.1:32872 (Press CTRL+C to quit)


[2025-08-17 00:29:42] INFO:     127.0.0.1:42354 - "GET /v1/models HTTP/1.1" 200 OK


[2025-08-17 00:29:42] INFO:     127.0.0.1:42368 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-17 00:29:42] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-17 00:29:44] INFO:     127.0.0.1:42374 - "POST /generate HTTP/1.1" 200 OK
[2025-08-17 00:29:44] The server is fired up and ready to roll!


In [6]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Llama-2-7b-chat-hf",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-08-17 00:29:47] Prefill batch. #new-seq: 1, #new-token: 16, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-08-17 00:29:47] INFO:     127.0.0.1:37964 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [7]:
terminate_process(server_process)

### EAGLE-2 Decoding via Frequency-Ranked Speculative Sampling

By employing a truncated high-frequency token vocabulary in the draft model, Eagle speculative decoding reduces `lm_head` computational overhead while accelerating the pipeline without quality degradation. For more details, checkout [the paper](https://arxiv.org/pdf/arXiv:2502.14856).

In our implementation, set `--speculative-token-map` to enable the optimization. You can get the high-frequency token in FR-Spec from [this model](https://huggingface.co/thunlp/LLaMA3-Instruct-8B-FR-Spec). Or you can obtain high-frequency token by directly downloading these token from [this repo](https://github.com/thunlp/FR-Spec/tree/main?tab=readme-ov-file#prepare-fr-spec-vocabulary-subset).

Thanks for the contribution from [Weilin Zhao](https://github.com/Achazwl) and [Zhousx](https://github.com/Zhou-sx). 

In [8]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Meta-Llama-3-8B-Instruct --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-LLaMA3-Instruct-8B --speculative-num-steps 5 \
    --speculative-eagle-topk 8 --speculative-num-draft-tokens 64 --speculative-token-map thunlp/LLaMA3-Instruct-8B-FR-Spec/freq_32768.pt \
    --mem-fraction 0.7 --cuda-graph-max-bs 2 --dtype float16 
"""
)

wait_for_server(f"http://localhost:{port}")

W0817 00:29:54.521000 2986029 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0817 00:29:54.521000 2986029 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-17 00:29:55] server_args=ServerArgs(model_path='meta-llama/Meta-Llama-3-8B-Instruct', tokenizer_path='meta-llama/Meta-Llama-3-8B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='127.0.0.1', port=32562, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='float16', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.7, max_running_requests=128, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, rand

[2025-08-17 00:29:56] Using default HuggingFace chat template with detected content format: string


W0817 00:30:02.994000 2986609 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0817 00:30:02.994000 2986609 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.
W0817 00:30:02.994000 2986610 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0817 00:30:02.994000 2986610 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-17 00:30:03] Casting torch.bfloat16 to torch.float16.


...

Capturing batches (bs=1 avail_mem=58.38 GB): 100%|██████████| 4/4 [00:06<00:00,  1.67s/it]
[2025-08-17 00:30:35] Capture draft cuda graph end. Time elapsed: 7.18 s. mem usage=0.24 GB. avail mem=58.33 GB.
[2025-08-17 00:30:35] Capture draft extend cuda graph begin. This can take up to several minutes. avail mem=58.33 GB


Capturing batches (bs=1 avail_mem=57.74 GB): 100%|██████████| 4/4 [00:00<00:00, 27.03it/s]
[2025-08-17 00:30:36] Capture draft extend cuda graph end. Time elapsed: 1.42 s. mem usage=0.59 GB. avail mem=57.74 GB.
[2025-08-17 00:30:36] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=128, context_len=8192, available_gpu_mem=57.74 GB


[2025-08-17 00:30:37] INFO:     Started server process [2986029]
[2025-08-17 00:30:37] INFO:     Waiting for application startup.
[2025-08-17 00:30:37] INFO:     Application startup complete.
[2025-08-17 00:30:37] INFO:     Uvicorn running on http://127.0.0.1:32562 (Press CTRL+C to quit)


[2025-08-17 00:30:37] INFO:     127.0.0.1:38546 - "GET /v1/models HTTP/1.1" 200 OK


[2025-08-17 00:30:38] INFO:     127.0.0.1:38550 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-17 00:30:38] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-08-17 00:30:38] INFO:     127.0.0.1:38564 - "POST /generate HTTP/1.1" 200 OK
[2025-08-17 00:30:38] The server is fired up and ready to roll!


In [9]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3-8B-Instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-08-17 00:30:42] Prefill batch. #new-seq: 1, #new-token: 17, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-17 00:30:42] INFO:     127.0.0.1:38572 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [10]:
terminate_process(server_process)

### EAGLE-3 Decoding

You can enable EAGLE-3 decoding by setting `--speculative_algorithm EAGLE3` and choosing an appropriate model.

In [11]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-3.1-8B-Instruct  --speculative-algorithm EAGLE3 \
    --speculative-draft-model-path jamesliu1/sglang-EAGLE3-Llama-3.1-Instruct-8B --speculative-num-steps 5 \
        --speculative-eagle-topk 8 --speculative-num-draft-tokens 32 --mem-fraction 0.6 \
        --cuda-graph-max-bs 2 --dtype float16
"""
)

wait_for_server(f"http://localhost:{port}")

W0817 00:30:49.243000 2989608 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0817 00:30:49.243000 2989608 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-17 00:30:50] server_args=ServerArgs(model_path='meta-llama/Llama-3.1-8B-Instruct', tokenizer_path='meta-llama/Llama-3.1-8B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='127.0.0.1', port=38160, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='float16', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.6, max_running_requests=128, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_see

[2025-08-17 00:30:51] Using default HuggingFace chat template with detected content format: string


W0817 00:30:57.456000 2990147 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0817 00:30:57.456000 2990147 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.
W0817 00:30:57.540000 2990148 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0817 00:30:57.540000 2990148 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-17 00:30:58] Casting torch.bfloat16 to torch.float16.


...

Capturing batches (bs=1 avail_mem=56.09 GB): 100%|██████████| 4/4 [00:05<00:00,  1.26s/it]
[2025-08-17 00:31:24] Capture draft cuda graph end. Time elapsed: 5.85 s. mem usage=-3.41 GB. avail mem=55.96 GB.
[2025-08-17 00:31:24] Capture draft extend cuda graph begin. This can take up to several minutes. avail mem=55.96 GB


Capturing batches (bs=1 avail_mem=55.85 GB): 100%|██████████| 4/4 [00:00<00:00, 92.86it/s]
[2025-08-17 00:31:24] Capture draft extend cuda graph end. Time elapsed: 0.62 s. mem usage=0.11 GB. avail mem=55.85 GB.
[2025-08-17 00:31:24] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=128, context_len=131072, available_gpu_mem=55.85 GB


[2025-08-17 00:31:25] INFO:     Started server process [2989608]
[2025-08-17 00:31:25] INFO:     Waiting for application startup.
[2025-08-17 00:31:25] INFO:     Application startup complete.
[2025-08-17 00:31:25] INFO:     Uvicorn running on http://127.0.0.1:38160 (Press CTRL+C to quit)


[2025-08-17 00:31:25] INFO:     127.0.0.1:36630 - "GET /v1/models HTTP/1.1" 200 OK


[2025-08-17 00:31:26] INFO:     127.0.0.1:36638 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-17 00:31:26] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-08-17 00:31:26] INFO:     127.0.0.1:36640 - "POST /generate HTTP/1.1" 200 OK
[2025-08-17 00:31:26] The server is fired up and ready to roll!


In [12]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-08-17 00:31:31] Prefill batch. #new-seq: 1, #new-token: 42, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-17 00:31:31] INFO:     127.0.0.1:36644 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [13]:
terminate_process(server_process)

## Multi Token Prediction

We support [MTP(Multi-Token Prediction)](https://arxiv.org/pdf/2404.19737) in SGLang by using speculative decoding. We use Xiaomi/MiMo-7B-RL model as example here (deepseek mtp usage refer to [deepseek doc](../references/deepseek.md#multi-token-prediction))

In [14]:
server_process, port = launch_server_cmd(
    """
    python3 -m sglang.launch_server --model-path XiaomiMiMo/MiMo-7B-RL --host 0.0.0.0 --trust-remote-code \
    --speculative-algorithm EAGLE --speculative-num-steps 1 --speculative-eagle-topk 1 --speculative-num-draft-tokens 2 \
    --mem-fraction 0.5
"""
)

wait_for_server(f"http://localhost:{port}")

W0817 00:31:38.902000 2993634 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0817 00:31:38.902000 2993634 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-17 00:31:40] server_args=ServerArgs(model_path='XiaomiMiMo/MiMo-7B-RL', tokenizer_path='XiaomiMiMo/MiMo-7B-RL', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=True, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='0.0.0.0', port=30114, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='auto', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.5, max_running_requests=128, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=647605639, constrained_jso

[2025-08-17 00:31:41] Using default HuggingFace chat template with detected content format: string


W0817 00:31:47.428000 2994428 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0817 00:31:47.428000 2994428 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.
W0817 00:31:47.558000 2994427 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0817 00:31:47.558000 2994427 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-17 00:31:49] Attention backend not explicitly specified. Use flashinfer backend by default.
[2025-08-17 00:31:49] Init torch distributed begin.


...

Capturing batches (bs=1 avail_mem=54.46 GB): 100%|██████████| 4/4 [00:02<00:00,  1.97it/s]
[2025-08-17 00:32:02] Capture draft cuda graph end. Time elapsed: 3.18 s. mem usage=0.89 GB. avail mem=54.46 GB.
[2025-08-17 00:32:02] Capture draft extend cuda graph begin. This can take up to several minutes. avail mem=54.46 GB


Capturing batches (bs=1 avail_mem=54.36 GB): 100%|██████████| 4/4 [00:00<00:00, 59.55it/s]
[2025-08-17 00:32:02] Capture draft extend cuda graph end. Time elapsed: 0.59 s. mem usage=0.09 GB. avail mem=54.36 GB.
[2025-08-17 00:32:02] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=128, context_len=32768, available_gpu_mem=54.36 GB


[2025-08-17 00:32:03] INFO:     Started server process [2993634]
[2025-08-17 00:32:03] INFO:     Waiting for application startup.
[2025-08-17 00:32:03] INFO:     Application startup complete.
[2025-08-17 00:32:03] INFO:     Uvicorn running on http://0.0.0.0:30114 (Press CTRL+C to quit)


[2025-08-17 00:32:03] INFO:     127.0.0.1:48862 - "GET /v1/models HTTP/1.1" 200 OK


[2025-08-17 00:32:04] INFO:     127.0.0.1:48870 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-17 00:32:04] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


In [15]:
import requests

url = f"http://localhost:{port}/v1/chat/completions"

data = {
    "model": "XiaomiMiMo/MiMo-7B-RL",
    "messages": [{"role": "user", "content": "What is the capital of France?"}],
}

response = requests.post(url, json=data)
print_highlight(response.json())

[2025-08-17 00:32:54] Prefill batch. #new-seq: 1, #new-token: 26, #cached-token: 0, token usage: 0.00, #running-req: 1, #queue-req: 0, 
[2025-08-17 00:32:54] INFO:     127.0.0.1:48878 - "POST /generate HTTP/1.1" 200 OK
[2025-08-17 00:32:54] The server is fired up and ready to roll!


[2025-08-17 00:32:55] Decode batch. #running-req: 1, #token: 94, token usage: 0.00, accept len: 1.67, cuda graph: True, gen throughput (token/s): 1.42, #queue-req: 0, 


[2025-08-17 00:32:56] Decode batch. #running-req: 1, #token: 167, token usage: 0.01, accept len: 1.82, cuda graph: True, gen throughput (token/s): 81.69, #queue-req: 0, 


[2025-08-17 00:32:57] Decode batch. #running-req: 1, #token: 236, token usage: 0.01, accept len: 1.73, cuda graph: True, gen throughput (token/s): 54.70, #queue-req: 0, 


[2025-08-17 00:32:58] Decode batch. #running-req: 1, #token: 305, token usage: 0.01, accept len: 1.73, cuda graph: True, gen throughput (token/s): 132.10, #queue-req: 0, 


[2025-08-17 00:32:59] Decode batch. #running-req: 1, #token: 380, token usage: 0.02, accept len: 1.88, cuda graph: True, gen throughput (token/s): 66.31, #queue-req: 0, 


[2025-08-17 00:32:59] INFO:     127.0.0.1:33118 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [16]:
terminate_process(server_process)

## References

EAGLE process is as follows:

- Within EAGLE the draft model predicts the next feature vector, i.e. the last hidden state of the original LLM, using the feature sequence $(f_1, ..., f_k)$ and the token sequence $(t_2, ..., t_{k+1})$. 
- The next token is then sampled from $p_{k+2}=\text{LMHead}(f_{k+1})$. Afterwards, the two sequences are extended in a tree style—branching out multiple potential continuations, with the branching factor per step controlled by the `speculative_eagle_topk` parameter—to ensure a more coherent connection of context, and are given as input again.
- EAGLE-2 additionally uses the draft model to evaluate how probable certain branches in the draft tree are, dynamically stopping the expansion of unlikely branches. After the expansion phase, reranking is employed to select only the top `speculative_num_draft_tokens` final nodes as draft tokens.
- EAGLE-3 removes the feature prediction objective, incorporates low and mid-layer features, and is trained in an on-policy manner.

This enhances drafting accuracy by operating on the features instead of tokens for more regular inputs and passing the tokens from the next timestep additionally to minimize randomness effects from sampling. Furthermore the dynamic adjustment of the draft tree and selection of reranked final nodes increases acceptance rate of draft tokens further. For more details see [EAGLE-2](https://arxiv.org/abs/2406.16858) and [EAGLE-3](https://arxiv.org/abs/2503.01840) paper.


For guidance how to train your own EAGLE model please see the [EAGLE repo](https://github.com/SafeAILab/EAGLE/tree/main?tab=readme-ov-file#train).